In [29]:
import tensorflow as tf
import math
import numpy as np
from nltk.tokenize import RegexpTokenizer

In [35]:
# bible
def sparse(x):
    t = [0] * vocab_size
    t[x] = 1
    return t
def process(x):
    t = [[w_i[i]] for i in x.split()]
    t = [sparse(x) for x in t]
    return t

pairs = np.load('pairs.npy')
w_i = np.load('word2index.npy').item()
i_w = np.load('index2word.npy')
vocab_size = len(i_w)
pairs.shape

(110506, 2)

In [30]:
input_size = vocab_size
print input_size
dims = [300, 200]
num_layers = 2

2999


In [31]:
def recurrence(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    z = tf.nn.sigmoid(tf.matmul(i, enc_layers[j]['Wxz']) + tf.matmul(p, enc_layers[j]['Whz']))
    r = tf.nn.sigmoid(tf.matmul(i, enc_layers[j]['Wxr']) + tf.matmul(p, enc_layers[j]['Whr']))
    h_ = tf.nn.tanh(tf.matmul(i, enc_layers[j]['Wxh']) +
                    tf.matmul(tf.mul(p, r), enc_layers[j]['Whh']))
    h = tf.mul(tf.sub(tf.ones_like(z), z), h_) + tf.mul(z, p)
    return tf.reshape(h, [dims[j-1]])

def recurrence2(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    z = tf.nn.sigmoid(tf.matmul(i, dec_layers[j]['Wxz']) + tf.matmul(p, dec_layers[j]['Whz']))
    r = tf.nn.sigmoid(tf.matmul(i, dec_layers[j]['Wxr']) + tf.matmul(p, dec_layers[j]['Whr']))
    h_ = tf.nn.tanh(tf.matmul(i, dec_layers[j]['Wxh']) +
                    tf.matmul(tf.mul(p, r), dec_layers[j]['Whh']))
    h = tf.mul(tf.sub(tf.ones_like(z), z), h_) + tf.mul(z, p)
    return tf.reshape(h, [dims[j-1]])

In [32]:
tf.reset_default_graph()
with tf.device('/cpu:0'):
    with tf.name_scope('encoder'):
        enc_layers = []
        incoming = tf.placeholder(shape=[None, input_size], dtype=tf.float32, name='encoder_incoming')
        enc_layers.append({'states': incoming, 'dim': input_size})
        
        for j in range(1, num_layers + 1):
            shape = [enc_layers[-1]['dim'], dims[j-1]]
            layer = dict()
            layer['Wxh'] = tf.Variable(tf.random_normal(
                shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whh'] = tf.Variable(tf.random_normal(
                [dims[j-1], dims[j-1]], stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
            
        # layer associated with update gate
            layer['Wxz'] = tf.Variable(tf.random_normal(shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whz'] = tf.Variable(tf.random_normal(shape=[
                dims[j-1], dims[j-1]], stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
        # layer associated with the reset gate
            layer['Wxr'] = tf.Variable(tf.random_normal(shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whr'] = tf.Variable(tf.random_normal(shape=[
                dims[j-1], dims[j-1]],
                stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
            layer['dim'] = dims[j-1]
            layer['initial'] = tf.placeholder(shape=[dims[j-1]], dtype=tf.float32, name='initial'+str(j))
            enc_layers.append(layer)
            layer['states'] = tf.scan(recurrence, enc_layers[-2]['states'], initializer=layer['initial'], name='encoder_states'+str(j))
            layer['last'] = tf.squeeze(tf.expand_dims(tf.nn.embedding_lookup(layer['states'], 
                                                                             tf.shape(layer['states'])[0] - 1), dim=0),
                                      name='last'+str(j))

In [33]:
with tf.device('/cpu:0'):
    with tf.name_scope('decoder'):
        
        dec_layers = []
        decoder_input = tf.placeholder(shape=[None, input_size], dtype=tf.float32)
        dec_layers.append({'states': decoder_input, 'dim': input_size})
        
        for j in range(1, num_layers + 1):
            shape = [dec_layers[-1]['dim'], dims[j-1]]
            layer = dict()
            layer['Wxh'] = tf.Variable(tf.random_normal(
                shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whh'] = tf.Variable(tf.random_normal(
                [dims[j-1], dims[j-1]], stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
            
        # layer associated with update gate
            layer['Wxz'] = tf.Variable(tf.random_normal(shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whz'] = tf.Variable(tf.random_normal(shape=[
                dims[j-1], dims[j-1]], stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
        # layer associated with the reset gate
            layer['Wxr'] = tf.Variable(tf.random_normal(shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whr'] = tf.Variable(tf.random_normal(shape=[
                dims[j-1], dims[j-1]],
                stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
            layer['dim'] = dims[j-1]
            dec_layers.append(layer)
            
            layer['states'] = tf.scan(recurrence2, dec_layers[-2]['states'], initializer=enc_layers[j]['last'], 
                                      name='decoder_states'+str(j))
            

        # softmax layer
        Why = tf.Variable(tf.random_normal(
            [dims[-1], input_size],
            stddev=1.0 / math.sqrt(dims[-1])), dtype=tf.float32)
        by = tf.Variable(tf.zeros(input_size), dtype=tf.float32)
        outputs = tf.matmul(dec_layers[-1]['states'], Why) + by
        
        # decoding step (with 'teacher forcing')

        decoder_target = tf.placeholder(shape=[None, input_size], dtype=tf.float32)
#         target = tf.squeeze(tf.nn.embedding_lookup(np.identity(input_size).astype(np.float32), decoder_target), 
#                             squeeze_dims=[1])
        
        
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(outputs, decoder_target))
        optimizer_op = tf.train.AdagradOptimizer(0.1).minimize(loss)

In [34]:
sess =  tf.Session()
sess.run(tf.initialize_all_variables())
ix = 0
print_step = 5000
avg_loss = 0

In [25]:
def generate_out(inp):
    inp = process(inp)
    out = ['START_TOKEN']
#     seed = [one_hot('eos')]
    seed = [[w_i['START_TOKEN']]]
    while True:
        feed = {incoming : inp, initial: np.zeros(dim), decoder_input: seed}
        o = sess.run(outputs, feed_dict=feed)[-1]
        o = np.argmax(o)
#         o = one_hot(id_token[o])
#         r = un_process(o)
        r = i_w[o]
        o = [o]
        seed.append(o)
        out.append(r)
        if r == 'END_TOKEN':
            break
    print ' '.join(out[1:-1])

In [ ]:
for i in range(pairs.shape[0]):
    if ix == len(pairs):
        print 'epoch ' + str(i + 1)
        ix = 0
    a = [sparse(x) for x in pairs[ix][0]]
    b = [sparse(x) for x in pairs[ix][1]]
#     a, b = np.array(pairs[ix][0]).reshape([-1, 1]), np.array(pairs[ix][1]).reshape([-1, 1])

    feed = {incoming: a, decoder_target: b[1:], decoder_input : b[:-1]}
    for j in enc_layers[1:]:
        feed[j['initial']] = np.zeros(j['dim'])
        
    l, _ = sess.run([loss, optimizer_op], feed_dict=feed)
    avg_loss += l
    if i % print_step == 0 and i != 0:
        print avg_loss / print_step
        avg_loss = 0
#         generate_out('god created man')
    ix += 1

In [26]:
generate_out('the night')

saying ,


In [10]:
w_i['START_TOKEN']

2996